Below code uses tensorflow 2.0.0-beta0 to classify fashion mnist data set using CNN

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
print(tf.__version__)

2.0.0-beta1


In [2]:
mnist = tf.keras.datasets.fashion_mnist   # Importing fashion mnist dataset
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()   #Loading train and test dataset

The first convolution expects a single tensor containing everything, so instead of 60,000 28x28x1 items in a list, we have a single 4D list that is 60,000x28x28x1, and the same for the test images.

In [3]:
class mycall(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')> 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training = True

callbacked = mycall()

The above implements callback which checks the accuracy at the end of each epoch and if the accuracy metric is
above threshold, the training stops thereby saving time.

In [4]:
#checking the shape and values of pixel of a training example

tf.shape(training_images)
print(tf.shape(training_images[0]))
print(training_images[0])


tf.Tensor([28 28], shape=(2,), dtype=int32)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   

In [5]:
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0

test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

'''The range of pixel size is from 0-255. It is better to normalize the data before proceeding. Normalizing the data improves the performance by removing unnecessary details.'''

'The range of pixel size is from 0-255. It is better to normalize the data before proceeding. Normalizing the data improves the performance by removing unnecessary details.'

Instead of the input layer at the top, a convolution layer is added before that so that only important features of 
images goes in the input layer.

Below are the parameters for conv2d func :

The number of convolutions to generate-64: this corresponds to different types of filter which might detect features vertically
, horizontally, in oblique angle etc.
The size of the Convolution or filter which is 3X3.
The activation function which outputs only positive value and negative value is made zero.
In the first layer, the shape of the input data.

Convolution is followed with a MaxPooling layer which  compresses the image, while maintaining the content of the features that were highlighted by the convolution. Here the pooling size is 2X2, the pool layer will go over each grid of 2X2 and pick the largest pixel value - quarter the size of the image. The idea is here to retain the important features. It repeats this across the image, and in so doing halves the number of horizontal, and halves the number of vertical pixels, effectively reducing the image by 25%.

Finally with another layer of convolution and pool, the image is flattened out for input.


In [6]:
model= tf.keras.Sequential([tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(28, 28, 1)),
                            tf.keras.layers.MaxPooling2D(2, 2),
                           # tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                           # tf.keras.layers.MaxPooling2D(2,2),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dense(10, activation='softmax')
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=10,callbacks=[callbacked])
print("Evaluating model on test data")
test_loss = model.evaluate(test_images, test_labels)

W0804 11:30:55.315113 15144 deprecation.py:323] From C:\Users\Gabriel\.conda\envs\tensorflow_env\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10816)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1384576   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 1,386,506
Trainable params: 1,386,506
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 0.3864 -